In [1]:
from bs4 import BeautifulSoup
from pymongo import MongoClient
import requests
import pandas as pd
import json
import re

In [2]:
client = MongoClient()
# Access/Initiate Database
db = client['Podcast']
# Access/Initiate Table
Podcasts = db['PodcastFinal']
#tab.insert_one( {'test': "item"})

In [3]:
Itunes_base_url = 'https://itunes.apple.com/ug/genre/podcasts/id26?mt=2'

In [4]:
Podcasts.count()

7

In [6]:
html = requests.get(Itunes_base_url)
soup = BeautifulSoup(html.text)

/Users/zephryin/anaconda/envs/py2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [7]:
df_genre = pd.DataFrame(columns=['Genre', 'Link'])
for i,genre in enumerate(soup.findAll(attrs={'class' : 'top-level-genre'})):
    temp = [str(genre.findAll(text=True)[0]), genre['href']]
    df_genre.loc[i] = temp

In [8]:
df_genre

,Genre,Link
0,Arts,https://itunes.apple.com/ug/genre/podcasts-art...
1,Business,https://itunes.apple.com/ug/genre/podcasts-bus...
2,Comedy,https://itunes.apple.com/ug/genre/podcasts-com...
3,Education,https://itunes.apple.com/ug/genre/podcasts-edu...
4,Games & Hobbies,https://itunes.apple.com/ug/genre/podcasts-gam...
5,Government & Organisations,https://itunes.apple.com/ug/genre/podcasts-gov...
6,Health,https://itunes.apple.com/ug/genre/podcasts-hea...
7,Kids & Family,https://itunes.apple.com/ug/genre/podcasts-kid...
8,Music,https://itunes.apple.com/ug/genre/podcasts-mus...
9,News & Politics,https://itunes.apple.com/ug/genre/podcasts-new...


In [9]:
df_subgenre = pd.DataFrame(columns=['Genre', 'SubGenre', 'Link'])
for genre in soup.findAll(attrs={'class' : 'list top-level-subgenres'}):
    for subgenre in genre.findAll('a'):
        subgroupName = str(subgenre.findAll(text=True)[0])
        link         = subgenre['href']
        m            = re.search('podcasts-(.+?)-', link.split('/')[-2])
        if m:
            main_genre   = m.group(1)
            main_genre   = main_genre.capitalize()
        else:
            main_genre   = subgroupName
        temp = [main_genre, subgroupName, link]
        df_subgenre.loc[len(df_subgenre)] = temp

In [10]:
df_subgenre.loc[29:35]['Genre'] = ['Religion & Spirituality']*7
df_subgenre.loc[36:38]['Genre'] = ['Science & Medicine']*3

/Users/zephryin/anaconda/envs/py2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/zephryin/anaconda/envs/py2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [44]:
failed = []

for p_url in df_subgenre.Link:
    soup2        = BeautifulSoup(requests.get(p_url).content)
    podcast_list = soup2.find(attrs={'class':'grid3-column'})
    for podcast in podcast_list.findAll('a'):
        podcast_name = podcast.findAll(text=True)[0] #str(podcast.findAll(text=True)[0])
        podcast_link = podcast['href']
    
        ##get Json
        podcast_id   = re.search('\/id(\d+)',podcast['href']).group(1)
        json_link    = 'https://itunes.apple.com/lookup?id=' + podcast_id
        try:
            j            = requests.get(json_link)
            j            = j.json()
            jsonfile     = j['results'][0]
            rss          = str(jsonfile['feedUrl'])
            #get link
            soup3        = BeautifulSoup(requests.get(rss).content)
            summaries = []
            for rss_summary  in soup3.findAll('itunes:summary'):
                #new = [ a for a in rss_summary]
                new = rss_summary.getText()
                summaries.extend(new)
            text = ' '.join(summaries)

            #place into mongoDB
            Podcasts.insert_one( {'PodcastName':podcast_name, 
                     'PodcastLink': podcast_link, 
                     'PodcastRSS' :text,
                     'PodcastJson': jsonfile})
        except:
            print (podcast_name + "Failed")
            failed.append(podcast_id)

言寺 midnight poemsFailed
峰哥Failed
"It's In A Book"Failed
Zhongwen PodcastFailed
2011 GLK Class How To VideosFailed
YogaFailed
Meditation and True RelaxationFailed
Beyond The PitchFailed
Rasika Digital Radio 2.0Failed


In [45]:
Podcasts.count()

1587

In [146]:
e = soup2.find(attrs={'class':'grid3-column'})
e1 = e.find('a')
e1

<a href="https://itunes.apple.com/ug/podcast/creative-pep-talk/id929743897?mt=2">Creative Pep Talk</a>

In [21]:
df_subgenre.Link[0:3]

0    https://itunes.apple.com/ug/genre/podcasts-art...
1    https://itunes.apple.com/ug/genre/podcasts-art...
2    https://itunes.apple.com/ug/genre/podcasts-art...
Name: Link, dtype: object

In [161]:
e1['href']

'https://itunes.apple.com/ug/podcast/creative-pep-talk/id929743897?mt=2'

In [171]:
re.search('\/id(\d+)',e1['href']).group(1)

'929743897'

In [167]:
m.group(1)

'929743897'

In [12]:
json_link = 'https://itunes.apple.com/lookup?id=929743897'
j = requests.get(json_link)

In [13]:
j = j.json()

In [14]:
jsonlink = j['results'][0]

In [18]:
str(jsonlink['feedUrl'])

'http://feeds.soundcloud.com/users/soundcloud:users:87660023/sounds.rss'

In [129]:
#from each link
get json from 'https://itunes.apple.com/lookup?id=929743897'
#from json
get 'rss'

In [19]:
rss = 'http://feeds.soundcloud.com/users/soundcloud:users:87660023/sounds.rss'

In [174]:
str(e1.findAll(text=True)[0])

'Creative Pep Talk'

In [217]:
text = []
for i  in soup3.findAll('itunes:summary'):
    new = [ a for a in i]
    text.extend(new)
final = ' '.join(text)

In [208]:
for i,ia in enumerate(i):
    print i, ia

0 Today we chat with Brandon Rike on why staying inspired is so important, and how to actually do it!

I love Brandon, he’s a good guy with a good soul. 

Brandon also has a podcast called ‘The Graphic Sound’. If you like CPT you might just like Brandon’s podcast too! This week we are both releasing episodes featuring each other. Go check out The Graphic Sound episode 10 to hear an in depth interview with ME.

http://www.thegraphicsound.com/10
http://www.thegraphicsound.com/
http://brandonrike.com/

Thanks to Yoni Wolf and his band WHY? for our theme music.

Thanks to Illustration Age for being our proud syndicate.
www.illustrationage.com

Thanks to Nate Utesch and Metavari for our other music! 
nthnl.com/
metavari.com/


In [218]:
tab.insert_one( {'PodcastName':str(e1.findAll(text=True)[0]), 
                 'PodcastLink': e1['href'], 
                 'PodcastRSS' :text,
                 'PodcastJson': j['results'][0]})

In [223]:
tab.find_one({})

{u'PodcastJson': {u'artistName': u'Andy J. Miller',
  u'artworkUrl100': u'http://is5.mzstatic.com/image/thumb/Music69/v4/76/12/e7/7612e730-9171-cabc-5ed1-e4c498786347/source/100x100bb.jpg',
  u'artworkUrl30': u'http://is5.mzstatic.com/image/thumb/Music69/v4/76/12/e7/7612e730-9171-cabc-5ed1-e4c498786347/source/30x30bb.jpg',
  u'artworkUrl60': u'http://is5.mzstatic.com/image/thumb/Music69/v4/76/12/e7/7612e730-9171-cabc-5ed1-e4c498786347/source/60x60bb.jpg',
  u'artworkUrl600': u'http://is5.mzstatic.com/image/thumb/Music69/v4/76/12/e7/7612e730-9171-cabc-5ed1-e4c498786347/source/600x600bb.jpg',
  u'collectionCensoredName': u'Creative Pep Talk',
  u'collectionExplicitness': u'cleaned',
  u'collectionHdPrice': 0,
  u'collectionId': 929743897,
  u'collectionName': u'Creative Pep Talk',
  u'collectionPrice': 0.0,
  u'collectionViewUrl': u'https://itunes.apple.com/us/podcast/creative-pep-talk/id929743897?mt=2&uo=4',
  u'contentAdvisoryRating': u'Clean',
  u'country': u'USA',
  u'currency': u'US

In [222]:
tab['PodcastJson']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'Podcast'), u'PodcastInfo.PodcastJson')